# Backprop thru RVQ

In [1]:
from typing import *

import torch
from matplotlib import pyplot as plt

import audiocraft
from audiocraft.models.encodec import HFEncodecCompressionModel
from transformers.models.encodec.modeling_encodec import EncodecEuclideanCodebook

from device import DEVICE

A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'
c:\Users\iGlop\anaconda3\envs\env_musicgen\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


We have CUDA.
gpu_name = 'NVIDIA GeForce RTX 3050 Ti Laptop GPU'


In [10]:
encodec: HFEncodecCompressionModel = audiocraft.models.encodec.CompressionModel.get_pretrained(
    'facebook/encodec_32khz', DEVICE, 
) # type: ignore

c:\Users\iGlop\anaconda3\envs\env_musicgen\Lib\site-packages\torch\nn\utils\weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [11]:
rvq = encodec.model.quantizer
type(rvq), rvq.num_quantizers

(transformers.models.encodec.modeling_encodec.EncodecResidualVectorQuantizer,
 4)

In [12]:
codebooks: List[EncodecEuclideanCodebook] = [x.codebook for x in rvq.layers]
type(codebooks[0]), [x.embed.shape for x in codebooks]    # 2048 codes of 128 dims

(transformers.models.encodec.modeling_encodec.EncodecEuclideanCodebook,
 [torch.Size([2048, 128]),
  torch.Size([2048, 128]),
  torch.Size([2048, 128]),
  torch.Size([2048, 128])])

In [22]:
print('RVQ codebooks norm:')
[torch.square(x.embed).sum(dim=1).sqrt().mean() for x in codebooks]

RVQ codebooks norm:


[tensor(35.7539, device='cuda:0'),
 tensor(23.9555, device='cuda:0'),
 tensor(22.8637, device='cuda:0'),
 tensor(19.0215, device='cuda:0')]